In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, basename
import pickle
from scipy.stats import pearsonr
from scipy.signal import periodogram, filtfilt, spectrogram, welch
import seaborn as sns
from tqdm import tqdm

from filtering import freq_filt

banner_coords = (44.3, -115.233)
res = pd.read_csv('/bsuscratch/zacharykeskinen/data/infrasound/eq_catalog/selected_v3.csv')
from shapely import wkt
res['geometry'] = res['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(res, geometry = 'geometry', crs = 'EPSG:4326')

result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/psd_results'
data_dir = '/bsuscratch/zacharykeskinen/data/infrasound/array_data'
with open(join(data_dir, 'merged/all_days'), 'rb') as f:
    days = pickle.load(f)
sps = 200

In [18]:
n = 0
avg_Pxx = np.array([])
all_psds = {}
for i, r in tqdm(res[res.analyze == 1].iterrows(), total = len(res[res.analyze == 1])):
    if i == i:
        dt = pd.to_datetime(r.time).strftime('%Y-%m-%d')
        day = days[dt]
        sig = {}
        s = pd.to_datetime(r.time)
        e = s + pd.Timedelta('7 second')
        try:
            if 0.33 in day.keys() and 1.33 in day.keys() and day['snotel']['Snow Depth (cm) Start of Day Values'] > 133:
                for name, fp in day.items():
                    if name != 'snotel':
                        series = pd.read_parquet(fp)[s:e]
                        arr = freq_filt(series.values.ravel(), 1, kind = 'highpass')
                        arr = arr[:7*200]
                        # f, Pxx = welch(arr, sps, scaling = 'density', window = 'hann')
                        # Pxx = filtfilt([1,1,1,1,1],5, Pxx)
                        sig[name] = arr
                df = pd.DataFrame(sig)
                indx = series.index[:7*200]
                df.index = indx
                if avg_Pxx.size == 0:
                    avg_Pxx = df
                else:
                    avg_Pxx = avg_Pxx + df
                    n +=1
                all_psds[r.time] = df
        except ValueError as e:
            print(dt)
            print(e)

with open(join(result_dir, 'eq_sigs.pkl'), 'wb') as f:
    pickle.dump(all_psds, f)

  5%|▌         | 2/38 [00:12<03:34,  5.96s/it]

2022-03-20
The length of the input vector x must be greater than padlen, which is 9.


100%|██████████| 38/38 [06:39<00:00, 10.52s/it]
